In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train_data= pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
pred_data=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
print(train_data.shape)
print(pred_data.shape)

In [ ]:
train_data.head(20)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from matplotlib import rcParams
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [ ]:
train_data.info()

In [ ]:
d = defaultdict(LabelEncoder)
sf_encode = train_data.apply(lambda x: d[x.name].fit_transform(x))
sf_encode = sf_encode.drop(['X', 'Y'], axis=1)

corrmat = sf_encode.corr()
f, ax = plt.subplots(figsize=(12, 12))
plot2 =sns.heatmap(corrmat, vmax=.8);
plt.xticks(rotation=90)
plt.yticks(rotation=45)
plot2.axes.set_title('Correlation Heat Map')
plt.show()

LOOKING AT THE CATEGORIES OF CRIME

In [ ]:
train_data.groupby("Category")["Category"].count().sort_values(ascending=False)

In [ ]:
import seaborn as sns
number_of_crimes = train_data.Category.value_counts()
_n_crime_plot = sns.barplot(x=number_of_crimes.index,y=number_of_crimes)
_n_crime_plot.set_xticklabels(number_of_crimes.index,rotation=90)


In [ ]:
train_data.groupby("PdDistrict")["PdDistrict"].count().sort_values(ascending=False)

LOOKING AT THE MOST DANGEROUS DISCTRICS

In [ ]:
most_dangerous_districts = train_data.PdDistrict.value_counts()
_n_crime_plot = sns.barplot(x=most_dangerous_districts.index,y=most_dangerous_districts)
_n_crime_plot.set_xticklabels(most_dangerous_districts.index,rotation=90)

HEAT MAP FOR NUMBER OF CRIMES PER DISTRICT

In [ ]:
pt = pd.pivot_table(train_data,index="PdDistrict",columns="Category",aggfunc=len,fill_value=0)["Dates"]
_ = pt.loc[most_dangerous_districts.index,number_of_crimes.index]
ax = sns.heatmap(_)
ax.set_title("Number of Crimes per District")

Adding Hours colum 

In [ ]:
train_data['Dates'] = train_data['Dates'].astype('datetime64[ns]')
train_data['Hour']= train_data.Dates.dt.hour


Visulaizing the Crimes wrt the hours in the day

In [ ]:
import pylab
import numpy as np

pylab.rcParams['figure.figsize'] = (14.0, 8.0)

larceny = train_data[train_data['Category'] == "LARCENY/THEFT"]
assault = train_data[train_data['Category'] == "ASSAULT"]
drug = train_data[train_data['Category'] == "DRUG/NARCOTIC"]
vehicle = train_data[train_data['Category'] == "VEHICLE THEFT"]
vandalism = train_data[train_data['Category'] == "VANDALISM"]
burglary = train_data[train_data['Category'] == "BURGLARY"]

with plt.style.context('fivethirtyeight'):
    ax1 = plt.subplot2grid((3,3), (0,0), colspan=3)
    ax1.plot(train_data.groupby('Hour').size(), 'ro-')
    ax1.set_title ('All crimes')
    start, end = ax1.get_xlim()
    ax1.xaxis.set_ticks(np.arange(start, end, 1))
    
    ax2 = plt.subplot2grid((3,3), (1, 0))
    ax2.plot(larceny.groupby('Hour').size(), 'o-')
    ax2.set_title ('Larceny/Theft')
    
    ax3 = plt.subplot2grid((3,3), (1, 1))
    ax3.plot(assault.groupby('Hour').size(), 'o-')
    ax3.set_title ('Assault')
    
    ax4 = plt.subplot2grid((3,3), (1, 2))
    ax4.plot(drug.groupby('Hour').size(), 'o-')
    ax4.set_title ('Drug/Narcotic')
    
    ax5 = plt.subplot2grid((3,3), (2, 0))
    ax5.plot(vehicle.groupby('Hour').size(), 'o-')
    ax5.set_title ('Vehicle')
    
    ax6 = plt.subplot2grid((3,3), (2, 1))
    ax6.plot(vandalism.groupby('Hour').size(), 'o-')
    ax6.set_title ('Vandalism')
    
    ax7 = plt.subplot2grid((3,3), (2, 2))
    ax7.plot(burglary.groupby('Hour').size(), 'o-')
    ax7.set_title ('Burglary')
  
    pylab.gcf().text(0.5, 1.03, 
                     'San Franciso Crime Occurence by Hour',
                     horizontalalignment='center',
                     verticalalignment='top', 
                     fontsize = 28)
    
plt.tight_layout(2)
plt.show()

Creating Months and Years Colum

In [ ]:
train_data['Dates'] = train_data['Dates'].astype('datetime64[ns]')
train_data['Month']= train_data.Dates.dt.month
train_data['Year']= train_data.Dates.dt.year


Showing Crimes with Months and Years

In [ ]:
pylab.rcParams['figure.figsize'] = (16.0, 5.0)
yearMonth = train_data.groupby(['Year','Month']).size()
ax = yearMonth.plot(lw=2)
plt.title('San Franciso Crimes Trend by Month&Year', fontsize=24)
plt.show()